# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [ ]:
# review the collections in our new database


In [6]:
# review a document in the establishments collection
establishments = db['establishments']
pprint(establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669df5415dc620f9474cbc94'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Greenwich High Road",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 8JQ",
    "Phone": "",
    "RatingValue": "",
    "LocalAuthorityBusinessID": "12345",
    "LocalAuthorityName": "Greenwich",
    "Geocode": {
        "Longitude": 0.0,
        "Latitude": 0.0
    },
    "RightToReply": "",
    "Distance": 0.0,
    "NewRatingPending": True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '123 Greenwich High Road',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Distance': 0.0,
 'Geocode': {'Latitude': 0.0, 'Longitude': 0.0},
 'LocalAuthorityBusinessID': '12345',
 'LocalAuthorityName': 'Greenwich',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE10 8JQ',
 'RatingValue': '',
 'RightToReply': '',
 '_id': ObjectId('669df8a29301cfa307b2b1dc')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

# Print the BusinessTypeID and BusinessType
pprint(business_type)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Extract the BusinessTypeID
business_type_id = business_type["BusinessTypeID"]

In [13]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)


In [14]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '123 Greenwich High Road',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 0.0,
 'Geocode': {'Latitude': 0.0, 'Longitude': 0.0},
 'LocalAuthorityBusinessID': '12345',
 'LocalAuthorityName': 'Greenwich',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE10 8JQ',
 'RatingValue': '',
 'RightToReply': '',
 '_id': ObjectId('669df8a29301cfa307b2b1dc')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")


Number of documents with LocalAuthorityName as 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})


In [18]:
# Check if any remaining documents include Dover
dover_count_after_deletion = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover' after deletion: {dover_count_after_deletion}")




Number of documents with LocalAuthorityName as 'Dover' after deletion: 0


In [19]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669df5415dc620f9474cbf7a'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289353',
       

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [24]:
# Change the data type from String to Decimal for longitude and latitude with error handling
establishments.update_many(
    {},
    [
        {"$set": {
            "Geocode.Latitude": {
                "$convert": {
                    "input": "$Geocode.Latitude",
                    "to": "double",
                    "onError": None,
                    "onNull": None
                }
            },
            "Geocode.Longitude": {
                "$convert": {
                    "input": "$Geocode.Longitude",
                    "to": "double",
                    "onError": None,
                    "onNull": None
                }
            }
        }}
    ]
)


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [27]:
# Change the data type from String to Integer for RatingValue with error handling
establishments.update_many(
    {"RatingValue": {"$ne": None}},
    [
        {"$set": {
            "RatingValue": {
                "$convert": {
                    "input": "$RatingValue",
                    "to": "int",
                    "onError": None,
                    "onNull": None
                }
            }
        }}
    ]
)


In [28]:
# Check that the coordinates and rating value are now numbers
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))


{'AddressLine1': '123 Greenwich High Road',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 0.0,
 'Geocode': {'Latitude': 0.0, 'Longitude': 0.0},
 'LocalAuthorityBusinessID': '12345',
 'LocalAuthorityName': 'Greenwich',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE10 8JQ',
 'RatingValue': None,
 'RightToReply': '',
 '_id': ObjectId('669df8a29301cfa307b2b1dc')}
